# Selenium

In [ ]:
# Python을 이용해서 웹 브라우저를 조작할 수 있는 자동화 프레임워크
!pip install selenium

In [ ]:
# 웹 브라우저를 제어할 수 있는 자동화 프레임워크-Chrome
!pip install webdriver-manager

In [ ]:
# 셀레니움 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import chormeDriverManager

In [ ]:
# 불러온 모듈 webdriver에서 Chrome() 객체를 생성합니다.
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# 스크래핑하려는 페이지의 요청을 보낸다.
## page_source 속성을 통해 Response의 HTML문서를 확인할 수 있음
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("http://www.example.com")
print(driver.page_source)

In [ ]:
## with as 구문 아래 실행문이 실행된 후 자동으로 종료 
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("http://www.example.com")
    print(driver.page_source)

In [ ]:
# By를 import해봅시다
from selenium.webdriver.common.by import By

In [ ]:
# p 태그에 해당하는 요소 하나를 찾아봅시다.
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("http://www.example.com")
    print(driver.find_element(By.TAG_NAME,"p").text)

In [ ]:
# p 태그에 해당하는 요소 여러개를 찾아봅시다.(반복문)
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("http://www.example.com")
    for element in driver.find_elements(By.TAG_NAME,"p"):
        print("Text:", element.text)

### Target: IndieStreet 이벤트 스크래핑

다음 사이트에 있는 행사의 이름들을 스크래핑 해봅시다 :
<br> https://indistreet.com/live?sortOption=startDate%3AASC 

해당 웹 페이지는 공연 리스트가 담긴 웹 페이지입니다.

In [ ]:
# 스크래핑에 필요한 라이브러리를 불러와봅시다.

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# 예시 사이트에 요청을 진행하고, 예시 사이트의 첫 번째 이벤트의 제목을 가져와봅시다.

## //*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[2]/div/a/div[2]/p[1]
## div의 인덱스는 1부터 시작

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

## 아래 실행문을 통해 웹 페이지가 비동기 처리되고 있음을 확인 가능(데이터 처리가 늦음)
## 이런 경우 requests를 쓰기 어려움
driver.get("https://indistreet.com/live?sortOption=startDate%3AASC")
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[2]/div/a/div[2]/p[1]').text

## Implicit Wait

In [ ]:
# 10초동안 Implicit Wait을 진행하도록 해서 스크래핑이 잘 이루어지도록 수정해봅시다.
# 요청이 완벽하게 응답이 되면 실행 or 10초를 기다림
from selenium.webdriver.support.ui import WebDriverWait
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://indistreet.com/live?sortOption=startDate%3AASC")
    driver.implicitly_wait(10) 
    print(driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[2]/div/a/div[2]/p[1]').text)

## Explicit Wait

In [ ]:
# 주어진 XPath가 등장할 때 까지 Wait을 진행해봅시다. (`EC.presence_of_element_located()` 활용)
# Explicit Wait를 활용해서 스크래핑이 잘 이루어지도록 코드를 작성해봅시다.
from selenium.webdriver.support import expected_conditions as EC

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://indistreet.com/live?sortOption=startDate%3AASC")
    # explicit wait으로 변경
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[2]/div/a/div[2]/p[1]')))

    print(element.text)

### 여러 공연 제목 가져오기

여러 공연의 제목들을 가져오기 위해 XPath를 관찰해봅시다.  
이 페이지의 공연 제목들에 해당하는 XPath는 다음과 같습니다:

```
//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[1]/div/a/div[2]/p[1]
//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[2]/div/a/div[2]/p[1]
//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[3]/div/a/div[2]/p[1]
...
```

이를 일반화해서 가장 먼저 등장하는 **10개**의 이름을 추출하는 코드를 작성

In [ ]:
# 10개의 이름을 스크래핑하는 코드 작성

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://indistreet.com/live?sortOption=startDate%3AASC")
    driver.implicitly_wait(10)

    for i in range(1, 11):
        element = driver.find_element(By.XPATH,f'//*[@id="__next"]/div/main/div[2]/div/div[4]/div[1]/div[{i}]/div/a/div[2]/p[1]')
        print(element.text)

### 마우스 이벤트 처리하기
#### 목표: 해시코드 사이트의 "로그인" 창 클릭하기
- https://hashcode.co.kr/

In [4]:
# 스크래핑에 필요한 라이브러리를 불러와봅시다.

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# 주어진 웹사이트를 누른 후, 우리가 원하는 버튼 요소를 찾은 후 마우스 이벤트를 실행시켜봅시다.

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
driver.get("https://hashcode.co.kr/")
driver.maximize_window()
driver.implicitly_wait(0.5)

xpath = "/html/body/div[1]/header/section/div/div/div/a[1]"
button = driver.find_element(By.CLASS_NAME,"UtilMenustyle__Link-sc-2sjysx-4.ewJwEL")
ActionChains(driver).click(button).perform()

## ActionChains 콤보, 동작들을 연쇄적으로 수행할 수 있는 객체
    
## 에러가 발생해서 확인해보니 CLASS이름이 바꼈음을 확인 가능
## CLASS이름을 변경해주어도 계속 에러가 발생
## 해당 사이트의 경우 브라우저가 최대화가 아니면 "로그인"버튼이 숨어버림
## 브라우저를 실행 후 최대화해서 로그인을 클릭할 수 있도록 코드 변경 - 정상 작동

### 키보드 이벤트 처리하기
#### 해시코드 사이트에 로그인하기
- https://hashcode.co.kr/

In [6]:
# 스크래핑에 필요한 라이브러리를 불러와봅시다.

from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [7]:
# driver를 이용해 해당 사이트에 요청을 보내봅시다.
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
driver.get("https://hashcode.co.kr/")
driver.maximize_window()
time.sleep(0.5)

# 내비게이션 바에서 "로그인" 버튼을 찾아 눌러봅시다.
xpath = "/html/body/div[1]/header/section/div/div/div/a[1]"
button = driver.find_element(By.CLASS_NAME,"UtilMenustyle__Link-sc-2sjysx-4.ewJwEL")
ActionChains(driver).click(button).perform()
time.sleep(0.5)

# 내 경우 깃허브 아이디를 사용하고 있음 한번 더 이동
xpath_github = '//*[@id="main-app-account"]/div/div[2]/div/div[2]/div[1]/div/div/footer/div[2]/form[3]/button'
btn_github = driver.find_element(By.XPATH,xpath_github)
ActionChains(driver).click(btn_github).perform()
time.sleep(0.5)

# "아이디" input 요소에 여러분의 아이디를 입력합니다.
id_input = driver.find_element(By.ID,"login_field")
ActionChains(driver).send_keys_to_element(id_input,"yoon29723106@gmail.com").perform()
time.sleep(0.5)

# "패스워드" input 요소에 여러분의 비밀번호를 입력합니다.
pw_input = driver.find_element(By.ID,"password")
ActionChains(driver).send_keys_to_element(pw_input,"duwns157").perform()
time.sleep(0.5)

# "로그인" 버튼을 눌러서 로그인을 완료합니다.
login_button = driver.find_element(By.CLASS_NAME,"btn.btn-primary.btn-block.js-sign-in-button")
ActionChains(driver).click(login_button).perform()